In [1]:
# Importing required libraries 
from keras.models import Sequential, Model, model_from_json
import matplotlib.pyplot as plt
import keras 
import pickle
import wave  # !pip install wave
import os
import pandas as pd
import numpy as np
import sys
import warnings
import librosa
import librosa.display
import IPython.display as ipd  # To play sound in the notebook

from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers

import sounddevice as sd

# ignore warnings 
if not sys.warnoptions:
    warnings.simplefilter("ignore")

# **1. Audio recording**

## **2. Data preparation and prediction**

In [2]:
# loading json and model architecture :
json_file = open('./model/model_json_1D.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("./model/model_1D.h5")
print("Loaded model from disk")

# the optimiser
opt = optimizers.RMSprop(learning_rate=0.00001, decay=1e-6)
loaded_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

Loaded model from disk


In [3]:
# Lets transform the recorded file so we can apply the predictions

X, sample_rate = librosa.load('./recordings/female_surprised.wav'
                              ,res_type='kaiser_fast'
                              ,duration=2.5
                              ,sr=44100
                              ,offset=0.5
                             )

sample_rate = np.array(sample_rate)
mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13),axis=0)
newdf = pd.DataFrame(data=mfccs).T
newdf

,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
0,-46.555317,-47.138985,-49.506126,-50.332485,-49.198311,-48.018578,-48.555622,-48.102818,-47.35228,-49.859734,...,-52.494144,-51.593437,-53.318974,-55.786404,-54.555508,-53.245281,-55.746098,-57.64333,-54.877254,-54.076523


In [4]:
# Apply predictions
newdf= np.expand_dims(newdf, axis=2)
newpred = loaded_model.predict(newdf, 
                         batch_size=16, 
                         verbose=1)

newpred

1/1 [==============================] - 0s 204ms/step


array([[0.0000000e+00, 1.5835106e-16, 1.3109989e-25, 2.9960216e-14,
        1.3988499e-03, 4.3415954e-10, 8.3324253e-10, 1.3181062e-30,
        1.9313556e-01, 7.2846872e-32, 5.8274738e-18, 8.0546558e-01,
        4.1610899e-13, 6.5885057e-23]], dtype=float32)

In [5]:
# Append the labels to it before we can make sense as to what it has predicted.

infile = open('./model/labels_1D','rb')
lb = pickle.load(infile)
infile.close()

# Get the final predicted label
final = newpred.argmax(axis=1)
final = final.astype(int).flatten()
final = (lb.inverse_transform((final)))
print(final) #emo(final) #gender(final) 

['male_neutral']
